In [3]:
import json
import collections
import os


from src.utils.metric_utils import compute_disentangle_score, compute_disentangle_scores_possible_empties


REPO_DIR = f"/workspace/ravel"
SRC_DIR = os.path.join(REPO_DIR, "src")
MODEL_DIR = os.path.join(REPO_DIR, "models")
DATA_DIR = os.path.join(REPO_DIR, "data")

tinyllama_dimension_to_log_path = {
    'SAE': {d: f'tinyllama-layer14-dim8192-reg0.5-ep5-sae-city_wikipedia_200k_dim{d}_3tok_Country.json'
            # Update the following dimensions to match your own results.
            # SAE might have different feature dimensions from run to run due to
            # randomness in the feature selection algorithm.
            for d in [71, 325, 399, 536, 8192]
    },
    # 'DAS': {d: f'tinyllama-layer14-dim{d}-baseline_daslora_3tok_Country_len48_pose_ep3_cause20000_evalall.json'
    #         for d in [16, 64]},
    # 'MDAS': {d: f'tinyllama-layer14-dim{d}-multitask_daslora_3tok_Country1_ex_Continent+Latitude+Longitude+Language+Timezone_len48_pose_ep3_cause20000_iso4000_evalall.json'
    #          for d in [16, 64]
    # },
}
# tinyllama_dimension_to_log_path['RandomSAE'] = {
#             64: f'tinyllama-layer14-dim8192-reg0.5-ep5-sae-city_wikipedia_200k_random_dim64_3tok_Country.json',
#             512: f'tinyllama-layer14-dim8192-reg0.5-ep5-sae-city_wikipedia_200k_random_dim512_3tok_Country.json'
#             }



entity_type = 'city'
target_attribute = 'Country'
split_type = 'context'
split_suffix = '-test'
model_name = 'tinyllama'


split_to_raw_example = json.load(
    open(os.path.join(DATA_DIR, model_name, f'{model_name}_{entity_type}_{split_type}_test.json')))
attribute_to_prompts = json.load(
    open(os.path.join(DATA_DIR, 'base', f'ravel_{entity_type}_attribute_to_prompts.json')))


attribute_to_iso_tasks = {
    a: [p + split_suffix for p in ps if p + split_suffix in split_to_raw_example]
    for a, ps in attribute_to_prompts.items() if a != target_attribute}
attribute_to_cause_tasks = {
    a: [p + split_suffix for p in ps if p + split_suffix in split_to_raw_example]
    for a, ps in attribute_to_prompts.items() if a == target_attribute}

print(attribute_to_iso_tasks)

for key in attribute_to_iso_tasks:
  print(key)

for key in attribute_to_cause_tasks:
  print(key, "Cause")


# method_to_data = collections.defaultdict(dict)
# for method in tinyllama_dimension_to_log_path:
#   for inv_dimension in tinyllama_dimension_to_log_path[method]:
#     log_data = json.load(
#         open(os.path.join(MODEL_DIR, tinyllama_dimension_to_log_path[method][inv_dimension])))

#     # print(log_data)

#     # for key in log_data:
#     #   print(key, log_data[key], "\n")

#     method_to_data[method][inv_dimension] = compute_disentangle_scores_possible_empties(
#         log_data, attribute_to_iso_tasks, attribute_to_cause_tasks)

{'Continent': [' "language": "English"}, {"city": "%s", "continent": "-test', '[{"city": "Los Angeles", "continent": "North America"}, {"city": "%s", "continent": "-test', 'city: %s, continent:-test'], 'Latitude': ['[{"city": "Paris", "lat": "48.9"}, {"city": "%s", "lat": "-test', '[{"city": "Paris", "lat": "49"}, {"city": "%s", "lat": "-test', '[{"city": "St. Petersburg", "lat": "59.9"}, {"city": "%s", "lat": "-test', '[{"city": "Tokyo", "lat": "36"}, {"city": "%s", "lat": "-test', 'city: %s, latitude: (-test'], 'Longitude': [' "lat": "37.7749° N"}, {"city": "%s", "long": "-test', '[{"city": "Kuala Lumpur", "long": "101.7"}, {"city": "%s", "long": "-test', '[{"city": "San Francisco", "long": "122.4"}, {"city": "%s", "long": "-test', 'city: %s, longitude: -test'], 'Language': [' "continent": "Asia"}, {"city": "%s", "language": "-test', 'People in Beijing speak Chinese. People in %s speak-test', 'People in Hong Kong speak Chinese, English. People in %s speak-test', "The city of %s's gov